## Start

In [ ]:
datadir = '../data';
wkdir = '../results';

mk_cd_dir(wkdir, false);
%imatlab_export_fig('print-png')

% loocv inputs

prepare_loocv = true

demo_loocv = true

demo_loocv_number_or_list = 'number' % 'number' or 'list'

demo_loocv_number_cmpds = 2

demo_loocv_list_cmpds = {'BRD-K04804440','BRD-K01507359','BRD-K87202646','BRD-K59853741', 'BRD-K27302037'} % Ciprofloxacin, Rifampin, Isoniazid, Q203, Thioacetazone

results_subdir_prefix = 'loocv_pcls/leave_out_cmpd_'

loocv_save_out = false % save tabular file for each treatment's PCL similarity score


% general inputs

rr_path = fullfile(datadir, 'replicate_correlation_all.txt')

fgr_path = fullfile(datadir, 'fraction_gr.txt')

% previous Spectral Clustering inputs

thrsh_rank = 20 % threshold for average pairwise rank of correlation across KABX to connect treatments as mutual nearest-neighbors

dynamic_thrsh_per_moa = false % if true then threshold is round(log(size of MOA) * thrsh_rank), otherwise identical threshold for every MOA

k_type = 'k_med_gap_den' % eigengap heuristic to take for estimating number of K clusters: k_num_zero, k_num_zero_plus_one, k_med_gap_den, k_gap_den (see create_laplacian_matrix.m for additional information)

if dynamic_thrsh_per_moa
    outdir_name = sprintf('pcls_spectral_clustering_thrsh_rank_le%dxlogsize_%s', thrsh_rank, k_type) % log(MOA size), i.e. the number of treatments/dsCGI profiles in the MOA
else
    outdir_name = sprintf('pcls_spectral_clustering_thrsh_rank_le%d_%s', thrsh_rank, k_type)
end

outdir = fullfile(wkdir, outdir_name)

mk_cd_dir(outdir, false);

In [5]:
exist(rr_path) > 0
exist(fgr_path) > 0


ans =

  logical

   1


ans =

  logical

   1


ans =

  logical

   1



## Add replicate reproducibility 

In [11]:
rr = rtable(rr_path);

Reading /idi/cgtb/morzech/idmp/combine_screens_456/replicate_correlation_all.txt



In [12]:
size(rr)
headt(rr)


ans =

       79879           5


ans =

  5x3 table

    idx           field                                     value                         
    ___    ____________________    _______________________________________________________

     1     {'count_replicates'}    {[                                                  2]}
     2     {'max_rcorr'       }    {[                                             0.2286]}
     3     {'rcorr'           }    {[                                             0.2286]}
     4     {'rcorr_rank'      }    {[                                             0.0264]}
     5     {'cid'             }    {'screen_bi1_tbda2:BRD-A00011930-001-01-7:50.000000uM'}



## Add fraction of GR

In [13]:
fgr = rtable(fgr_path);

Reading /idi/cgtb/morzech/idmp/combine_screens_456/fraction_gr.txt



In [14]:
size(fgr)
headt(fgr)


ans =

       79873           4


ans =

  4x3 table

    idx             field                                 value                   
    ___    ________________________    ___________________________________________

     1     {'cid'                 }    {'kabx2:BRD-A01514287-001-01-8:0.097500uM'}
     2     {'x_median_total_count'}    {[                                 396223]}
     3     {'sum_gr_le0_30'       }    {[                                      0]}
     4     {'frac_gr_le0_30'      }    {[                                      0]}



In [15]:
rr.cid(~ismember(rr.cid, fgr.cid))


ans =

  6x1 cell array

    {'screen4wk_screen5wk_kabx2_tbda1:DMSO:0uM'}
    {'screen5_wk:DMSO:0uM'                     }
    {'screen5cp:DMSO:0uM'                      }
    {'screen6:DMSO:0.000000uM'                 }
    {'screen_bi1_tbda2:DMSO:0uM'               }
    {'screen_bi2_kabxlow:DMSO:0uM'             }



In [16]:
fgr.cid(~ismember(fgr.cid, rr.cid))


ans =

  0x1 empty cell array



## Full model section

In [6]:
ls(outdir)

ds_corr_n7225x9427.gctx
ds_corr_rank_n7225x9427.gctx
pcl_median_corr_n9427x1889.gctx
pcl_median_corr_rank_div_group_size_n9427x1889.gctx
pcl_median_corr_rank_n9427x1889.gctx



### Add replicate reproducibility and fraction of GR to existing GCTs

In [17]:
% Create gcts
fields = {'pcl_similarity_score','pcl_and_moa_agree'};

for ii = 1:numel(fields)
    disp(fields{ii})
    g = glob(fullfile(outdir,strcat(fields{ii},'_n*.gctx')))
    s = parse_gctx(g{1});
    s.chd
    s.cid(~ismember(s.cid, rr.cid))
    s.cid(~ismember(s.cid, fgr.cid))
    s = annotate_ds(s,table2struct(rr(:,{'cid','rcorr','rcorr_rank'})),'dim','column','keyfield','cid');
    s = annotate_ds(s,table2struct(fgr(:,{'cid','x_median_total_count','frac_gr_le0_30'})),'dim','column','keyfield','cid');
    s.chd
    
    mkgctx(fullfile(outdir,strcat(fields{ii})), s)
end

median_corr

g =

  1x1 cell array

    {'/idi/cgtb/abond/combine_screens_456/pcls_for_manuscript_final/validate_new_pcls_thrsh_rank_le20_k_med_gap_den/pcls_all_no_loo_scoring_exclude_self/pcl_median_corr_n79873x1704.gctx'}

Reading /idi/cgtb/abond/combine_screens_456/pcls_for_manuscript_final/validate_new_pcls_thrsh_rank_le20_k_med_gap_den/pcls_all_no_loo_scoring_exclude_self/pcl_median_corr_n79873x1704.gctx [1704x79873]
Done [6.27 s].

ans =

  4x1 cell array

    {'broad_id'  }
    {'pert_dose' }
    {'pert_id'   }
    {'pert_idose'}


ans =

  0x1 empty cell array


ans =

  0x1 empty cell array


ans =

  8x1 cell array

    {'x_median_total_count'}
    {'frac_gr_le0_30'      }
    {'rcorr'               }
    {'rcorr_rank'          }
    {'broad_id'            }
    {'pert_dose'           }
    {'pert_id'             }
    {'pert_idose'          }

Saving HDF5 dataset to: /idi/cgtb/abond/combine_screens_456/pcls_for_manuscript_final/validate_new_pcls_thrsh_rank_le20_k_med_gap_den

In [18]:
s.chd
s.rhd


ans =

  8x1 cell array

    {'x_median_total_count'}
    {'frac_gr_le0_30'      }
    {'rcorr'               }
    {'rcorr_rank'          }
    {'broad_id'            }
    {'pert_dose'           }
    {'pert_id'             }
    {'pert_idose'          }


ans =

  5x1 cell array

    {'num_pert_id_unique'}
    {'pcl_broad_id'      }
    {'pcl_desc'          }
    {'pcl_pert_id'       }
    {'pcl_size'          }



## LOOCV section

In [ ]:
if prepare_loocv

    unique_kabx_cmpds_tbl = rtable(unique_kabx_cmpds_tbl_path);

    size(unique_kabx_cmpds_tbl)
    headt(unique_kabx_cmpds_tbl)
    
    unique_kabx_cmpds_list = unique(unique_kabx_cmpds_tbl.kabx_cmpd);

    length(unique_kabx_cmpds_list)
    
    number_of_cmpds_loocv = length(unique_kabx_cmpds_list)
    
    if demo_loocv
       if strcmp(demo_loocv_number_or_list, 'number')
           number_of_cmpds_loocv = max(1, demo_loocv_number_cmpds)
           
           index_cmpds_loocv = 1:number_of_cmpds_loocv
           
       elseif strcmp(demo_loocv_number_or_list, 'list')
           number_of_cmpds_loocv = length(demo_loocv_list_cmpds)
           
           index_cmpds_loocv = find(ismember(unique_kabx_cmpds_list, demo_loocv_list_cmpds))'
       else
           error('Invalid input for demo_loocv_number_or_list: number or list')
       end
    end
    
    for i = index_cmpds_loocv
    
        % If the current iteration number is a multiple of 50
        if mod(i, 50) == 0
            % Print a status message
            fprintf('Currently at iteration %d\n', i);
        end

        leave_out_cmpd = unique_kabx_cmpds_list(i);
        
        loo_wkdir = fullfile(wkdir, strcat(results_subdir_prefix, strjoin(unique_kabx_cmpds_list(i))));

        mk_cd_dir(loo_wkdir, false);
        
        loo_outdir = fullfile(loo_wkdir, outdir_name)
        
        mk_cd_dir(loo_outdir, false);
        
        % step specific commands
    
        ls(loo_outdir)
        
        % Add replicate reproducibility and fraction of GR to existing GCTs
        
        % Create gcts
        fields = {'pcl_similarity_score','pcl_and_moa_agree'};

        for ii = 1:numel(fields)
            disp(fields{ii})
            g = glob(fullfile(loo_outdir,strcat(fields{ii},'_n*.gctx')))
            s = parse_gctx(g{1});
            s.chd
            s.cid(~ismember(s.cid, rr.cid))
            s.cid(~ismember(s.cid, fgr.cid))
            s = annotate_ds(s,table2struct(rr(:,{'cid','rcorr','rcorr_rank'})),'dim','column','keyfield','cid');
            s = annotate_ds(s,table2struct(fgr(:,{'cid','x_median_total_count','frac_gr_le0_30'})),'dim','column','keyfield','cid');
            s.chd

            mkgctx(fullfile(loo_outdir,strcat(fields{ii})), s)
        end
        
        s.chd
        s.rhd

    end
    
end